## Data Mining  Assignment 1  - Predictive Modelling - Kavya Reddy Basupalli 

### Understanding Scoring Metric

Now that we have prepared our data and pre-processed it,its time for us to understand the business problem in depth and its related scoring metric. According to our business problem, our predictive model tries to find the severity of the Erythemato-squamous diseases in its initial stage.

Lets define few metrics,

*True Positive= Our model predicts the severity of the disease, but in reality our prediction comes to be true so there is no harm on either end.

*False Positive = Our model predicts the severity of the disease, but in reality our prediction comes false so now the money spent for its diagnosis would be considered as loss.

*True Negative= Our model predicts no severity of disease, but in reality our prediction is true so there is no loss.

*False Negative= Our model predicts no severity of disease, but in reality our prediction is false, in that case human with those symptoms will not get diagnosed and it leads to serious health issues.



So I feel False Negative has major impact on human's health and life because it effects them mentally and physically.
Now, since False Negative is considered 'Recall' is the best score.

### Step 1:  Import Libraries and Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
import warnings
warnings.filterwarnings("ignore")
from sklearn.neural_network import MLPClassifier

np.random.seed(1)

### Step 2 : Load the data

In [2]:
df_bin= pd.read_csv('C:/Users/Kavya Reddy Basupall/Downloads/disease_train_df.csv')
df_bin

,ERYTHEMA,SCALING,DEFINITE_BORDERS,ITCHING,KOEBNER_PHENOMENON,POLYGONAL_PAPULES,FOLLICULAR_PAPULES,ORAL_MUCOSAL_INVOLVEMENT,KNEE_AND_ELBOW_INVOLVEMENT,SCALP_INVOLVEMENT,...,DISAPPEARANCE_OF_THE_GRANULAR_LAYER,VACUOLISATION_AND_DAMAGE_OF_BASAL_LAYER,SPONGIOSIS,SAW-TOOTH_APPEARANCE_OF_RETES,FOLLICULAR_HORN_PLUG,PERIFOLLICULAR_PARAKERATOSIS,INFLAMMATORY_MONOLUCLEAR_INFLITRATE,BAND-LIKE_INFILTRATE,AGE,CLASS_CODE
0,2,2,2,0,0,0,0,0,2,2,...,3,0,0,0,0,0,3,0,36,1
1,2,2,1,2,0,0,0,0,0,0,...,0,0,2,0,0,0,1,0,33,2
2,2,2,2,0,0,0,0,0,3,0,...,2,0,0,0,0,0,2,0,23,1
3,1,1,0,1,1,0,0,0,0,0,...,0,0,2,0,0,0,2,0,60,4
4,2,3,2,0,0,0,0,0,3,2,...,0,0,0,0,0,0,1,0,70,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,3,2,2,0,2,0,0,0,3,2,...,0,0,0,0,0,0,2,0,60,1
246,2,2,2,0,1,0,0,0,0,0,...,0,0,2,0,0,0,2,0,22,4
247,2,2,2,1,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,33,2
248,2,2,2,1,0,0,2,0,2,2,...,0,0,2,0,2,2,2,0,7,6


### Step3 : Loading our Previous trained data 

In [3]:
train_X=pd.read_csv('C:/Users/Kavya Reddy Basupall/Downloads/disease_train_X.csv')
train_y=pd.read_csv('C:/Users/Kavya Reddy Basupall/Downloads/disease_train_y.csv')
test_X=pd.read_csv('C:/Users/Kavya Reddy Basupall/Downloads/disease_test_X.csv')
test_y=pd.read_csv('C:/Users/Kavya Reddy Basupall/Downloads/disease_test_y.csv')

### Step 4: Model the data 

In [4]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

### Step 4.1 Logistic Regression using random and grid search

In [5]:
score_measure = "recall"
kfolds = 5

param_grid = {'C':[0.01,0.1,1,2,5], # C is the regulization strength
               'penalty':['l1', 'l2','elasticnet','none'],
              'solver':['saga','liblinear'],
              'max_iter': np.arange(250,500)
                  
}

log_reg = LogisticRegression()
rand_search = RandomizedSearchCV(estimator =log_reg, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1  # n_jobs=-1 will utilize all available CPUs 
                                )

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

best_log_reg = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is nan
... with parameters: {'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 494, 'C': 5}


In [6]:
score_measure = "recall"
kfolds = 5
best_penality = rand_search.best_params_['penalty']
best_solver = rand_search.best_params_['solver']
min_regulization_strength=rand_search.best_params_['C']
min_iter = rand_search.best_params_['max_iter']

#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    
    'C':np.arange(min_regulization_strength-0.05,min_regulization_strength+0.05), 
               'penalty':[best_penality],
              'solver':[best_solver],
              'max_iter': np.arange(min_iter-300,min_iter+300)
}

log_reg =  LogisticRegression()
grid_search = GridSearchCV(estimator = log_reg, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1 # n_jobs=-1 will utilize all available CPUs 
                )

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

best_log_reg = grid_search.best_estimator_

Fitting 5 folds for each of 600 candidates, totalling 3000 fits
The best recall score is nan
... with parameters: {'C': 4.95, 'max_iter': 194, 'penalty': 'l1', 'solver': 'liblinear'}


In [7]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
Recall_logistic= {TP/(TP+FN)}

Accuracy=1.0000000 Precision=1.0000000 Recall=1.0000000 F1=1.0000000


In [8]:
performance = pd.concat([performance, pd.DataFrame({'model':"logistic using random & grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Neural Network With RandomizedSearchCV

In [9]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'solver': 'adam', 'max_iter': 5000, 'learning_rate_init': 0.01, 'learning_rate': 'constant', 'hidden_layer_sizes': (60, 40, 20), 'alpha': 0, 'activation': 'tanh'}
CPU times: total: 1.06 s
Wall time: 25.2 s


In [10]:
%%time
y_pred = bestRecallTree.predict(test_X)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        34
           2       1.00      0.92      0.96        13
           3       1.00      1.00      1.00        24
           4       0.92      1.00      0.96        11
           5       1.00      1.00      1.00        20
           6       1.00      1.00      1.00         6

    accuracy                           0.99       108
   macro avg       0.99      0.99      0.99       108
weighted avg       0.99      0.99      0.99       108

CPU times: total: 0 ns
Wall time: 19.3 ms


### Neural Network With GridSearchCV

In [11]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (30,), (50,), (70,), (90,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [.5, .7, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [0.005, 0.01, 0.15],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
{'activation': 'tanh', 'alpha': 0.5, 'hidden_layer_sizes': (30,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.01, 'max_iter': 5000, 'solver': 'adam'}
CPU times: total: 1.08 s
Wall time: 19.5 s


In [12]:
%%time
y_pred = bestRecallTree.predict(test_X)

print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        34
           2       0.93      1.00      0.96        13
           3       1.00      1.00      1.00        24
           4       1.00      0.91      0.95        11
           5       1.00      1.00      1.00        20
           6       1.00      1.00      1.00         6

    accuracy                           0.99       108
   macro avg       0.99      0.98      0.99       108
weighted avg       0.99      0.99      0.99       108

CPU times: total: 0 ns
Wall time: 10 ms


###  Step 4.2 SVM Model using random and grid search

In [13]:
score_measure = "recall"
kfolds = 3

param_grid = {'C':np.arange(0.1,100,10),  #  regularization parameter.
               'kernel':['linear', 'rbf','poly'],
              'gamma':['scale','auto'],
              'degree':np.arange(1,10), #degree is for the polynomial kernal
              'coef0':np.arange(1,10) #coef0 is for the polynomial kernal
                  
}

svc = SVC()
rand_search = RandomizedSearchCV(estimator =svc, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1  # n_jobs=-1 will utilize all available CPUs 
                                )

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

best_svc = rand_search.best_estimator_

Fitting 3 folds for each of 500 candidates, totalling 1500 fits
The best recall score is nan
... with parameters: {'kernel': 'linear', 'gamma': 'scale', 'degree': 5, 'coef0': 5, 'C': 40.1}


In [14]:
score_measure = "recall"
kfolds = 3
best_kernel = rand_search.best_params_['kernel']
best_gamma = rand_search.best_params_['gamma']
min_regulization=rand_search.best_params_['C']
best_degree = rand_search.best_params_['degree']
best_coef0=rand_search.best_params_['coef0']

#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    
    'C':np.arange(min_regulization,min_regulization+3), 
               'kernel':[best_kernel],
              'gamma':[best_gamma],
              'degree': np.arange(best_degree-1,best_degree+1),
            'coef0': np.arange(best_coef0-3,best_coef0+3)
}

svm_grid =  SVC()
grid_search = GridSearchCV(estimator = svm_grid, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1 # n_jobs=-1 will utilize all available CPUs 
                )

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

best_svm = grid_search.best_estimator_

Fitting 3 folds for each of 36 candidates, totalling 108 fits
The best recall score is nan
... with parameters: {'C': 40.1, 'coef0': 2, 'degree': 4, 'gamma': 'scale', 'kernel': 'linear'}


In [15]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
Recall_SVM = {TP/(TP+FN)}

Accuracy=1.0000000 Precision=1.0000000 Recall=1.0000000 F1=1.0000000


In [16]:
performance = pd.concat([performance, pd.DataFrame({'model':"svm using Random & Grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Step 4.3 Decision Tress using random search and grid search

In [17]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,100),  
    'min_samples_leaf': np.arange(1,100),
    'min_impurity_decrease': np.arange(0.0001, 0.0005),
    'max_leaf_nodes': np.arange(5, 100), 
    'max_depth': np.arange(1,25), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

best_DTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is nan
... with parameters: {'min_samples_split': 34, 'min_samples_leaf': 28, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 65, 'max_depth': 10, 'criterion': 'gini'}


In [18]:
score_measure = "recall"
kfolds = 5
min_samples_split = rand_search.best_params_['min_samples_split']
min_samples_leaf = rand_search.best_params_['min_samples_leaf']
min_impurity_decrease = rand_search.best_params_['min_impurity_decrease']
max_leaf_nodes = rand_search.best_params_['max_leaf_nodes']
max_depth = rand_search.best_params_['max_depth']
criterion = rand_search.best_params_['criterion']
#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    'min_samples_split': np.arange(min_samples_split-2,min_samples_split+2),  
    'min_samples_leaf': np.arange(min_samples_leaf-2,min_samples_leaf+2),
    'min_impurity_decrease': np.arange(min_impurity_decrease-0.0001, min_impurity_decrease+0.0001, 0.00005),
    'max_leaf_nodes': np.arange(max_leaf_nodes-2,max_leaf_nodes+2), 
    'max_depth': np.arange(max_depth-2,max_depth+2), 
    'criterion': [criterion]
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

best_DTree = grid_search.best_estimator_

Fitting 5 folds for each of 1024 candidates, totalling 5120 fits
The best recall score is nan
... with parameters: {'criterion': 'gini', 'max_depth': 8, 'max_leaf_nodes': 63, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 26, 'min_samples_split': 32}


In [19]:
c_matrix = confusion_matrix(test_y, grid_search.predict(test_X))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")
Recall_Dtree= {TP/(TP+FN)}

Accuracy=1.0000000 Precision=1.0000000 Recall=1.0000000 F1=1.0000000


In [20]:
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Step 5: Performance of models

In [21]:
performance

,model,Accuracy,Precision,Recall,F1
0,logistic using random & grid search,1.0,1.0,1.0,1.0
0,svm using Random & Grid search,1.0,1.0,1.0,1.0
0,Decision Tree,1.0,1.0,1.0,1.0


In [22]:
performance.sort_values(by=['Recall'])

,model,Accuracy,Precision,Recall,F1
0,logistic using random & grid search,1.0,1.0,1.0,1.0
0,svm using Random & Grid search,1.0,1.0,1.0,1.0
0,Decision Tree,1.0,1.0,1.0,1.0


### Summary 
We have performed different modelling techniques to dermatology data and observed the scoring metric values to be 100%. 
However, after adding neural network using random and grid search, the accuracy turned to show 0.99 which is similar to our data whose scoring metric values using all logistic, decision and SVM models showed better performance.
Finally, SVM, Decision Tree and Logistic modelling technique is more efficient as they show 100% accuracy than the neural network.

